# 1. Preliminaries 

Importing libraries

In [1]:
# general management
import os, glob, chardet, json  

# manage data
import pandas as pd
import numpy as np

# nlp libraries
import re, nltk
nltk.download('stopwords') # for removing stopwords
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer # for stemming

# pre-processing data
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# bayesian modeling
import stan

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jriveraespejo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


loading data

In [2]:
path = '/home/jriveraespejo/Desktop/project_europa/data/final/'
full_path = path + "7_info_match_round3_complete.csv"        
df = pd.read_csv(full_path, sep=',')

outlook the variables of interest

In [3]:
print(df['Primary Industry description'])

0      Producers of commercial, military and private ...
1      Producers of commercial, military and private ...
2      Producers of commercial, military and private ...
3      Producers of commercial, military and private ...
4      Producers of commercial, military and private ...
                             ...                        
436    Owners and operators of highways, rail tracks,...
437    Producers and distributors of electric power, ...
438    Providers of oil and gas drilling and related ...
439                                                  NaN
440                                                  NaN
Name: Primary Industry description, Length: 441, dtype: object


replacing NaN's with empty string

In [4]:
df['Primary Industry description'] = df['Primary Industry description'].fillna(' ')

In [5]:
print(df['Primary Industry description'])

0      Producers of commercial, military and private ...
1      Producers of commercial, military and private ...
2      Producers of commercial, military and private ...
3      Producers of commercial, military and private ...
4      Producers of commercial, military and private ...
                             ...                        
436    Owners and operators of highways, rail tracks,...
437    Producers and distributors of electric power, ...
438    Providers of oil and gas drilling and related ...
439                                                     
440                                                     
Name: Primary Industry description, Length: 441, dtype: object


# 2. Text cleaning

## 2.1 creating corpus

standardizing data

In [6]:
corpus = []

for i in range(df.shape[0]):
    
    # preliminary
    IndDesc = re.sub('[^a-zA-Z]', ' ', df['Primary Industry description'][i]) # replace 'punctuation' with ' '
    IndDesc = IndDesc.lower()
    IndDesc = IndDesc.split()
    
    # remove stopwords and stemming process
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not') # do not remove 'not'
    
    # don't apply stemming to stopwords
    ps = PorterStemmer()
    IndDesc = [ ps.stem(word) for word in IndDesc if not word in set(all_stopwords) ] 
    
    # join all words again
    IndDesc = ' '.join(IndDesc)
    corpus.append(IndDesc)

In [7]:
len(corpus)

441

In [8]:
print(corpus)

['produc commerci militari privat aircraft spacecraft includ manufactur militari equip vehicl explos ordnanc guidanc system artilleri ammunit relat weaponri includ also manufactur satellit', 'produc commerci militari privat aircraft spacecraft includ manufactur militari equip vehicl explos ordnanc guidanc system artilleri ammunit relat weaponri includ also manufactur satellit', 'produc commerci militari privat aircraft spacecraft includ manufactur militari equip vehicl explos ordnanc guidanc system artilleri ammunit relat weaponri includ also manufactur satellit', 'produc commerci militari privat aircraft spacecraft includ manufactur militari equip vehicl explos ordnanc guidanc system artilleri ammunit relat weaponri includ also manufactur satellit', 'produc commerci militari privat aircraft spacecraft includ manufactur militari equip vehicl explos ordnanc guidanc system artilleri ammunit relat weaponri includ also manufactur satellit', 'manufactur automobil light truck motorcycl well 

## 2.2 creating bag of words

we keep a maximum number of features of 250, as we only have 441 observations

In [9]:
cv = CountVectorizer(max_features = 250)

In [10]:
cv.fit_transform(corpus)

<441x250 sparse matrix of type '<class 'numpy.int64'>'
	with 5933 stored elements in Compressed Sparse Row format>

observe the 250 features used based on the CountVectorizer

In [11]:
dic_names = cv.get_feature_names()
dic_names

['aberti',
 'accessori',
 'accord',
 'activ',
 'advertis',
 'agenc',
 'agricultur',
 'air',
 'airport',
 'alcohol',
 'also',
 'america',
 'applic',
 'apprais',
 'automobil',
 'automot',
 'bank',
 'base',
 'beer',
 'beverag',
 'brewer',
 'broadcast',
 'brokerag',
 'build',
 'builder',
 'busi',
 'capit',
 'carbon',
 'carrier',
 'case',
 'casualti',
 'cement',
 'center',
 'chang',
 'chemic',
 'circular',
 'civil',
 'classifi',
 'co',
 'combin',
 'commerci',
 'commod',
 'compani',
 'compon',
 'compris',
 'concret',
 'conglomer',
 'construct',
 'consult',
 'consum',
 'contribut',
 'conveni',
 'cool',
 'criteria',
 'crush',
 'dam',
 'data',
 'develop',
 'dialysi',
 'distil',
 'distributor',
 'district',
 'domin',
 'drug',
 'econom',
 'economi',
 'effici',
 'electr',
 'electron',
 'employ',
 'energi',
 'engag',
 'engin',
 'ensur',
 'entertain',
 'entiti',
 'environ',
 'environment',
 'equip',
 'equiti',
 'estat',
 'exclud',
 'financi',
 'firm',
 'fix',
 'flavor',
 'food',
 'footprint',
 'foss

In [12]:
W = cv.fit_transform(corpus).toarray()
G = df.iloc[:, -1].values

Notice that the vectorized form of the 250 features (words), is made by word frequencies. It would be more convenient to convert it to relative frequencies.

In [13]:
print( W.shape )

(441, 250)


In [14]:
for i in range(W.shape[0]):
    print( W[i,] )

[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 2 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 2 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0

 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0]
[3 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 1 2 0 0 0 0 0 1 0 2 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 2 0 0 0 5 0 0 0 0 0 0 1 2 2 0 0 0 0 0 0 1
 0 0 2 3 0 0 0 0 0 0 0 0 0 2 0 0 0 0 2 3 3 0 0 0 2 1 0 0 0 0 1 0 0 0 0 0 0
 2 0 0 0 0 3 1 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 3 0 0 1 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 2 0 0 0 2 0 0 0 0
 4 0 0 0 0 2 0 0 0 0 0 1 0 0 3 0 3 0 0 0 0 0 0 2 0 0 0 2 0 0 0 0 0 0 0 0 0
 5 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[3 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 1 2 0 0 0 0 0 1 0 2 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 2 0 0 0 5 0 0 0 0 0 0 1 2 2 0 0 0 0 0 0 1
 0 0 2 3 0 0 0 0 0 0 0 0 0 2 0 0 0 0 2 3 3 0 0 0 2 1 0 0 0 0 1 0 0 0 0 0 0
 2 0 0 0 0 3 1 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 3 0 0 1 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 2 0 0 0 2 0 0 0 0
 4 0 0 0 0 2 0 0 0 0 0 1 0 0 3 0 3 0 0 0 0 0 0 2 0 0 0 2 0 

relative frequency conversion

In [15]:
W = np.array(W)
W = W / W.sum(axis=1).reshape((441,1)) # '0' will result in 'nan'
W = np.nan_to_num(W, copy=True, nan=0.0, posinf=None, neginf=None) # replace 'nan' by '0'
W = np.round_(W, decimals = 3)

<ipython-input-15-8e56dfa94f7d>:2: RuntimeWarning: invalid value encountered in true_divide
  W = W / W.sum(axis=1).reshape((441,1)) # '0' will result in 'nan'


In [16]:
for i in range(W.shape[0]):
    print( W[i,] )

[0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.067 0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.067 0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.067 0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.133 0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.133 0.    0.    0.    0.    0.
 0.133 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0

 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
[0.    0.    0.    0.    0.    0.    0.167 0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.167 0.
 0.    0.    0.    0.    0.    0.167 0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0. 

 0.    0.    0.    0.    0.    0.    0.062 0.    0.    0.   ]
[0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.062 0.    0.    0.    0.    0.    0.    0.125 0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.062 0.    0.062 0.    0.125 0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.062 0.    0.062 0.062 0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.062 0.    0.
 0.    0.    0.    0.    0. 

 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
[0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.091 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.091 0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.091 0.    0.
 0.    0.    0.    0.    0.    0.091 0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.091 0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0. 

saving the data

In [17]:
df_worldcloud = pd.DataFrame(W, columns=dic_names)
df_worldcloud['G'] = G
df_worldcloud['Name'] = df['Name_1']

fullpath = path + '8_NLPdata.csv'
df_worldcloud.to_csv(fullpath, index=False, header=True, sep=',')

For the 'green' variable we notice we have no issues

In [18]:
G.shape

(441,)

In [19]:
print(G)

[0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1
 1 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0
 1 0 0 1 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 1 0 1 1 1 0 0 0 1 0 0 1 1 0 1
 0 0 0 0 0 0 1 0 0 1 1 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 0 1 1 1 1]


## 2.3 preparing data

In [20]:
W_train, W_test, G_train, G_test = train_test_split(W, G, test_size = 0.10, random_state = 0)

In [21]:
print(W_train.shape)

(396, 250)


In [22]:
print(G_train.shape)

(396,)


# 3. Modeling: Word cloud model

## 3.1 Full Bayesian

### 3.1.1 fitting the model

defining the data as a list

HERE I HAVE A PROBLEM AS THE W_train HAS TO BE JSON, AND I DO NOT SEEM TO GRASP HOW TO DO IT.
see:
- https://github.com/stan-dev/pystan/issues/51
- https://github.com/stan-dev/httpstan/blob/main/httpstan/utils.py

I continue this in `R`

In [23]:
W_train_list = W_train.tolist()
# have to convert it to JSON (dictionary)

In [24]:
nlp_data = {
    "N": W_train.shape[0],
    "K": W_train.shape[1],
    "G": list(G_train),
    "W": W_train_JSON
}

NameError: name 'W_train_JSON' is not defined

In [ ]:
print(nlp_data)

defining the bayesian model

In [ ]:
mcmc_code = """
data {
    int N;
    int K;
    int G[N];
    matrix[N, k] W;
}
parameters {
    vector[K] b;
}
model {
    vector[N] v; // linear predictor
    vector[N] p; // probability
    
    // priors
    b ~ normal(0, 0.2); // highly regularizing prior
    
    // model
    v = W * b;
    p = inv_logit(v);
    G ~ bernoulli(p);
}
"""

fitting the model

In [ ]:
posterior = stan.build(mcmc_code, data=nlp_data, random_seed=1)

In [ ]:
fit = posterior.sample(num_chains=4, num_samples=2000)

check the results

In [ ]:
fit_df = fit.to_frame()
print(fit_df.describe().T)

### 3.1.2 posterior predictive

function to produce the mean prediction

In [ ]:
def bayes_MeaPred(X, fit_frame):
    Y_pred = np.dot(X, fit_frame)
    return(Y_pred)

calculate the mean prediction

In [ ]:
G_pred = bayes_MeaPred(X=W_test, fit_frame=fit_df)
print(np.concatenate((G_pred.reshape(len(G_pred), 1), G_test.reshape(len(G_test),1)), 1))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(G_test, G_pred)
print(cm)
accuracy_score(G_test, G_pred)